In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC

In [2]:
from scipy.io import loadmat
from scipy.io import savemat
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist

In [10]:
mat = loadmat('..\\results\\var_k\dblp\\res_dblp_exp_2_002_128.mat')
features_matrix = mat['embs']

mat = loadmat("../data/dblp.mat")
A = mat["network"]
labels_matrix = mat["group"]
labels_count = labels_matrix.shape[1]

groups = labels_matrix.toarray()
A = A.toarray()
X = features_matrix

print(groups[:5, :5])
print(A[:5, :5])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [11]:
# Number of nodes
num_nodes = 13326
num_features = 128

In [12]:
X_train, X_test, A_train, A_test = train_test_split(X, A, test_size=0.25, random_state=2137)

In [13]:
from tensorflow.keras.metrics import Metric, Precision, Recall

class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()
        self.f1_score = self.add_weight(name="f1", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
        p = self.precision.result()
        r = self.recall.result()
        self.f1_score.assign(2 * ((p * r) / (p + r + 1e-6)))  # Added a small epsilon to avoid division by zero

    def result(self):
        return self.f1_score

    def reset_state(self):
        self.precision.reset_states()
        self.recall.reset_states()
        self.f1_score.assign(0)


In [14]:
model = Sequential([
    Dense(64, activation='relu', input_dim=num_features),
    Dense(64, activation='relu'),
    Dense(num_nodes, activation='sigmoid')  # Output layer
])

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[
        BinaryAccuracy(name='accuracy'),
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='auc'),
        F1Score()
    ])

In [16]:
model.fit(X_train, A_train, epochs=50, batch_size=10, validation_split=0.1)

Epoch 1/50
900/900 [==============================] - 9s 9ms/step - loss: 0.0188 - accuracy: 0.9980 - precision: 3.0939e-04 - recall: 0.0013 - auc: 0.5817 - f1_score: 4.9915e-04 - val_loss: 0.0041 - val_accuracy: 0.9996 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5755 - val_f1_score: 0.0000e+00
Epoch 2/50
900/900 [==============================] - 7s 8ms/step - loss: 0.0035 - accuracy: 0.9996 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5647 - f1_score: 0.0000e+00 - val_loss: 0.0034 - val_accuracy: 0.9996 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5568 - val_f1_score: 0.0000e+00
Epoch 3/50
900/900 [==============================] - 7s 8ms/step - loss: 0.0032 - accuracy: 0.9996 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5481 - f1_score: 0.0000e+00 - val_loss: 0.0033 - val_accuracy: 0.9996 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5378 - val_f1_score: 0.0000e+00
Epoch 4/50
900/900 [==================

In [17]:
loss, accuracy, _, _, _, _ = model.evaluate(X_test, A_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

105/105 [==============================] - 2s 14ms/step - loss: 0.0054 - accuracy: 0.9996 - precision: 0.3771 - recall: 0.2153 - auc: 0.6934 - f1_score: 0.2741
Test Loss: 0.005420168861746788, Test Accuracy: 0.9995616674423218


In [28]:
def predict_adjacency_row(model, node_features):
    """
    Predict the adjacency row for a given node's features using a trained neural network model.
    
    Parameters:
    - model: The trained neural network model.
    - node_features: The feature vector for the node of interest (as a numpy array).
    
    Returns:
    - A binary vector representing the predicted connections from the node to all other nodes.
    """
    # Ensure node_features is the right shape: (1, num_features)
    node_features = np.array(node_features).reshape(1, -1)
    
    # Predict the connection probabilities using the model
    predicted_probabilities = model.predict(node_features)
    
    # Convert probabilities to binary values (0 or 1) using a threshold (e.g., 0.5)
    predicted_connections = (predicted_probabilities > 0.5).astype(int)
    
    # Return the first (and only) row of the predictions
    return predicted_connections[0]

# Example usage:
# Assuming you have the features for a specific node:
node_index = 10  # For example, if you want to predict connections for the 10th node
node_features = X[node_index]  # Extract features for this node

predicted_row = predict_adjacency_row(model, node_features)
print("Predicted connections for node:\n")
for i in range(13326):
    if predicted_row[i] > 0:
        print(i)

print("\nReal\n")
for i in range(13326):
    if A[node_index, i] > 0:
        print(i)

1/1 [==============================] - 0s 21ms/step
Predicted connections for node:

697
847
1870
2158
2791
3047
3067
3912
4257
4258
8522

Real

697
719
847
857
1869
1870
2158
2791
3047
3067
3068
3912
4257
4258
6549
8522


In [46]:
mat = loadmat('..\\results\\var_k\\blogcatalog\\res_blogcatalog_exp_4_002_128.mat')
features_matrix = mat['embs']

mat = loadmat("../data/blogcatalog.mat")
A = mat["network"]
labels_matrix = mat["group"]
labels_count = labels_matrix.shape[1]

groups = labels_matrix.toarray()
A = A.toarray()
X = features_matrix
num_nodes = 10312
num_features = 128

In [47]:
X_train, X_test, A_train, A_test = train_test_split(X, A, test_size=0.25, random_state=2137)

In [48]:
model = Sequential([
    Dense(64, activation='relu', input_dim=num_features),
    Dense(64, activation='relu'),
    Dense(num_nodes, activation='sigmoid')  # Output layer
])

In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[
        BinaryAccuracy(name='accuracy'),
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='auc'),
        F1Score()
    ])

In [50]:
model.fit(X_train, A_train, epochs=50, batch_size=10, validation_split=0.1)

Epoch 1/50
696/696 [==============================] - 6s 7ms/step - loss: 0.0548 - accuracy: 0.9918 - precision: 0.0121 - recall: 0.0040 - auc: 0.8705 - f1_score: 0.0060 - val_loss: 0.0247 - val_accuracy: 0.9937 - val_precision: 0.7852 - val_recall: 0.0088 - val_auc: 0.9283 - val_f1_score: 0.0175
Epoch 2/50
696/696 [==============================] - 5s 7ms/step - loss: 0.0244 - accuracy: 0.9939 - precision: 0.6727 - recall: 0.0198 - auc: 0.9255 - f1_score: 0.0384 - val_loss: 0.0243 - val_accuracy: 0.9937 - val_precision: 0.6067 - val_recall: 0.0265 - val_auc: 0.9285 - val_f1_score: 0.0508
Epoch 3/50
696/696 [==============================] - 5s 7ms/step - loss: 0.0240 - accuracy: 0.9939 - precision: 0.6434 - recall: 0.0299 - auc: 0.9265 - f1_score: 0.0572 - val_loss: 0.0238 - val_accuracy: 0.9937 - val_precision: 0.6319 - val_recall: 0.0284 - val_auc: 0.9307 - val_f1_score: 0.0544
Epoch 4/50
696/696 [==============================] - 5s 7ms/step - loss: 0.0237 - accuracy: 0.9939 - prec

In [51]:
loss, accuracy, _, _, _, _ = model.evaluate(X_test, A_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

81/81 [==============================] - 1s 12ms/step - loss: 0.0234 - accuracy: 0.9939 - precision: 0.6342 - recall: 0.1360 - auc: 0.9260 - f1_score: 0.2240
Test Loss: 0.0233688335865736, Test Accuracy: 0.9939163327217102
